In [1]:
import urllib3
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#전처리가 완료된 데이터 불러오기 
df = pd.read_csv('cleaned_cheeseball.csv')
df.drop([df.columns[0]],axis=1, inplace=True)
df.drop([df.columns[0]],axis=1, inplace=True)
df.drop([df.columns[0]],axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True)
df

,index,cleaned_content
0,0,TV PDBHC 주가 치즈볼 긴장 전국 치즈볼 마르 영상 대박
1,1,TV PDBHC 주가 치즈볼 긴장 치킨 프랜차이즈 치즈볼 자체 생산 외주 통해 제품...
2,2,TV PDBHC 주가 치즈볼 긴장 컨텐츠 우리나라 대기업 여러가지 집중 중소기업 지면
3,3,TV PDBHC 주가 치즈볼 긴장 바삭 푸드 썬리취 마니커 크림
4,4,TV PDBHC 주가 치즈볼 긴장 최종 평가
...,...,...
18238,18774,연휴 멕시 카나 치킨 아주 매콤 뿌리 고치 쫀득 더블 치즈볼 이면
18239,18775,배달의민족 순수 치킨 할인
18240,18776,금요일 배달의민족 순수 치킨 할인
18241,18777,또래오래 우촌 배달의민족 쿠폰 프로모션 치킨 금토일 치킨 파티 일요일 배민 또래오래


In [3]:
#onesentence만들기
one_sentence=''
for i in range(len(df)):
    one_sentence += (' '+ df['cleaned_content'][i])


In [4]:
#글자수 제한때문에 onesentence 를 9500단어씩 나눠서 API를 사용해야함 
#9500단위로 나눌거니까 전체길이/9500
print(len(one_sentence))
quot = int((len(one_sentence)/9500)+1)
print(quot)

1466154
155


In [5]:
#a에 빈리스트를 quot개만큼. 빈리스트마다 9500개씩 넣을거임 ex. a = [ 9500개데이터, 9500개데이터2, ...]
a = []
for n in range(quot):
    a.append([])


In [6]:
for n in range(quot):
    a[n] = one_sentence[n*9500:(n+1)*9500]




In [7]:
#데이터가 a[0]~a[154]에 들어감 마지막에는 나머지 값만 
print(len(a[153]))


9500


In [8]:
#b에 빈리스트를 a개만큼. 빈리스트마다 a->api적용값 넣을거임 
b = []
for n in range(quot):
    b.append([])

In [9]:
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
 
accessKey = "285a98de-eb7a-4292-a3e0-d36f5219be67"
analysisCode = "ner" #객체명 인식 API 사용 

In [10]:
#9500단어씩 나누어진 문장을 단어 토큰화 및 객체명 (음식,음료 등) 으로 카테고리화하여 저장

for i in range(len(a)):
    text = a[i]
    requestJson = {"access_key": accessKey, "argument": {"text": text,"analysis_code": analysisCode}}
    http = urllib3.PoolManager()
    response = http.request("POST",openApiURL,headers={"Content-Type": "application/json; charset=UTF-8"},body=json.dumps(requestJson))
    res_data = str(response.data,"utf-8")

    json_load = json.loads(res_data)

    b[i] = pd.DataFrame(json_load['return_object']['sentence'][0]['NE'])




In [11]:
c = []
for n in range(quot):
    c.append([])
    c[n] = b[n]


In [12]:
#b값 안전하게 c값으로 옮김 
df1 = c

In [13]:
#나누었던 데이터를 병합 
df1 = pd.concat([c[0],c[1]], axis=0, ignore_index=True)
df1

,id,text,type,begin,end,weight,common_noun
0,0.0,TV,TMI_HW,0.0,0.0,0.229573,0.0
1,1.0,PDBHC,TMI_HW,1.0,1.0,0.095379,0.0
2,2.0,치즈볼,CV_FOOD,3.0,4.0,0.634702,0.0
3,3.0,치즈볼,CV_FOOD,7.0,8.0,0.347629,0.0
4,4.0,TV PDBHC,TMI_HW,13.0,14.0,0.082489,0.0
...,...,...,...,...,...,...,...
180,84.0,떡볶이,CV_FOOD,2153.0,2154.0,1.000000,0.0
181,85.0,떡볶이,CV_FOOD,2263.0,2264.0,1.000000,0.0
182,86.0,떡볶이,CV_FOOD,2363.0,2364.0,1.000000,0.0
183,87.0,떡볶이,CV_FOOD,2441.0,2442.0,1.000000,0.0


In [15]:
df1 = c[0]

In [16]:
for i in range(1,len(c)):
    
    df2 =c[i]
    df1 = pd.concat([df1,df2], ignore_index=True)
df1

,id,text,type,begin,end,weight,common_noun
0,0.0,TV,TMI_HW,0.0,0.0,0.229573,0.0
1,1.0,PDBHC,TMI_HW,1.0,1.0,0.095379,0.0
2,2.0,치즈볼,CV_FOOD,3.0,4.0,0.634702,0.0
3,3.0,치즈볼,CV_FOOD,7.0,8.0,0.347629,0.0
4,4.0,TV PDBHC,TMI_HW,13.0,14.0,0.082489,0.0
...,...,...,...,...,...,...,...
16052,92.0,음악,FD_ART,899.0,900.0,0.362715,0.0
16053,93.0,라이프,OGG_ECONOMY,911.0,913.0,0.904651,0.0
16054,94.0,떡볶이,CV_FOOD,1237.0,1238.0,1.000000,0.0
16055,95.0,떡볶이,CV_FOOD,1841.0,1842.0,1.000000,0.0


In [17]:
#카테고리화된 단어들 중 필요한 카테고리 ex) 음료 의 단어들만을 추출 
texts = []
for a in range(len(df1)):
    if df1['type'][a] == 'CV_DRINK': #주제
        texts.append(df1['text'][a])

In [18]:
texts

['맥주',
 '블랙 프라이 데잌 gomshushu BHC 곰슈슈',
 '맥주',
 '맥주',
 '술',
 '술',
 '술',
 '맥주',
 'Fanta',
 '맥주',
 '맥주',
 '맥주',
 '맥주',
 '맥주',
 '맥주',
 '맥주',
 '치킨',
 '콜라',
 '사이다',
 '와인',
 '맥주',
 '치킨 콜라',
 '커피',
 '쥬스',
 '요거트',
 '콜라 블랙알리오',
 '주 스무디',
 '맥주',
 '맥주 치즈볼',
 '쥬스',
 '요거트',
 '요거트',
 '맥주',
 '치킨 콜라',
 '맥주',
 '맥주',
 '맥주',
 '생맥주',
 '생맥주',
 '칵테일',
 '칵테일',
 '와인',
 '맥주',
 '얼그레이 크림',
 '맥주',
 '커피',
 '콜라',
 '카페라떼',
 '와인',
 '모닝커피 주스',
 '맥주',
 '술',
 '맥주',
 '맥주',
 '소주',
 '프라푸치노',
 '추핫',
 '코코넛 밀크',
 '와인',
 '와인',
 '와인',
 '와인',
 '보드카',
 '술',
 '술',
 '맥주',
 '맥주',
 '맥주 치즈볼',
 '음료',
 '음료',
 '콜라',
 '맥주',
 '와인',
 '와인',
 '카페라떼',
 '맥주 안주 크림 치즈볼',
 '술',
 '콜라',
 '카페라떼 초코',
 '맥주',
 '치킨 콜라 치즈볼',
 '술안주 치밥',
 '와인',
 '맥주',
 '와인',
 '소주',
 '술',
 'coffee americano',
 '맥주',
 '맥주',
 '콜라',
 'Wine',
 '맥주',
 '코카콜라 치킨',
 '맥주',
 '맥주',
 '백피',
 '조니워커',
 '조니워커',
 '조니워커',
 '맥주',
 '커피',
 '맥주',
 '맥주 치즈볼',
 '코카콜라 후라이드 치킨',
 '맥주',
 '맥주',
 '맥주',
 '술',
 '소주',
 '맥주',
 '술',
 '맥주',
 '맥주',
 '아메리카노',
 '아메리카노',
 '커피 기프티콘',
 '아메리카노 기프티콘',
 '술',
 '와인',
 '레드

In [19]:
df_final = pd.DataFrame({'drink':texts}) #주제별로 컬럼명 변경 

In [20]:
df_final.to_csv('./drink_categorization_data.csv')